In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import folium
from folium import IFrame
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/School/DataMining/CW_accidents/'
CW_DATA_FILENAME = 'AccidentLondonBoroughs2223.csv'
CW_MYDATA_FILENAME = 'my_borough.csv'
MY_LOCAL_AUTHORITY_HIGHWAY = 'E09000021'

def dataframe_from_csv(filename: str) -> pd.DataFrame:
    return pd.read_csv(os.path.join(FOLDER_PATH,filename))

def filter_my_dataset(local_authority_highway: str) -> pd.DataFrame:
    df_all_data = dataframe_from_csv(CW_DATA_FILENAME)
    return df_all_data[df_all_data['Local_Authority_Highway'] == MY_LOCAL_AUTHORITY_HIGHWAY].copy()

In [ ]:
df_main = dataframe_from_csv(CW_MYDATA_FILENAME)
#Use this to filter new dataframe (with unneccessary columns removed)
df_main = df_main.iloc[:, [3,4,5,6,7,8,9,10,11,14,16,17,18,19,20,24,25,26]]
df_main.head()

,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Local_Authority_Highway,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions
0,169840,-0.250492,51.414457,1,3,1,1,13/01/2005,5,E09000021,3,6,50,0,-1,0,4,1
1,165370,-0.304009,51.375047,1,3,3,1,18/01/2005,3,E09000021,3,3,50,5,4,0,1,1
2,163980,-0.290258,51.362347,1,2,2,2,09/01/2005,1,E09000021,0,3,30,3,4,0,4,1
3,167190,-0.264617,51.390837,1,3,3,1,23/01/2005,1,E09000021,3,3,50,0,-1,0,4,1
4,172110,-0.256614,51.434962,1,2,2,1,21/01/2005,6,E09000021,3,3,30,0,-1,0,4,1


In [ ]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Location_Northing_OSGR                   2070 non-null   int64  
 1   Longitude                                2070 non-null   float64
 2   Latitude                                 2070 non-null   float64
 3   Police_Force                             2070 non-null   int64  
 4   Accident_Severity                        2070 non-null   int64  
 5   Number_of_Vehicles                       2070 non-null   int64  
 6   Number_of_Casualties                     2070 non-null   int64  
 7   Date                                     2070 non-null   object 
 8   Day_of_Week                              2070 non-null   int64  
 9   Local_Authority_Highway                  2070 non-null   object 
 10  1st_Road_Number                          2070 no

In [ ]:
for idx in [2,3,7,9,10,12,13,14,15,16,17]:
    df_main[df_main.columns[idx]] = df_main[df_main.columns[idx]].astype('category')

df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   Location_Northing_OSGR                   2070 non-null   int64   
 1   Longitude                                2070 non-null   float64 
 2   Latitude                                 2070 non-null   category
 3   Police_Force                             2070 non-null   category
 4   Accident_Severity                        2070 non-null   int64   
 5   Number_of_Vehicles                       2070 non-null   int64   
 6   Number_of_Casualties                     2070 non-null   int64   
 7   Date                                     2070 non-null   category
 8   Day_of_Week                              2070 non-null   int64   
 9   Local_Authority_Highway                  2070 non-null   category
 10  1st_Road_Number                     

In [1]:
df_main['Date'] = pd.to_datetime(df_main['Date'], dayfirst=True, format='%d%m%Y')

NameError: name 'pd' is not defined

In [ ]:
categorical_columns = df_main.select_dtypes(include=['category']).columns.tolist()
numeric_columns = df_main.select_dtypes(include=['number']).columns.tolist()

In [ ]:
df_main[numeric_columns].describe()

,Location_Northing_OSGR,Longitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Road_Type
count,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000
mean,167668.603865,-0.286730,2.835749,1.867150,1.245411,4.112560,5.075845
std,2212.555004,0.019886,0.389665,0.699536,0.643716,1.868408,1.650035
min,159950.000000,-0.327183,1.000000,1.000000,1.000000,1.000000,1.000000
25%,166210.000000,-0.303106,3.000000,1.000000,1.000000,3.000000,3.000000
50%,168025.000000,-0.291780,3.000000,2.000000,1.000000,4.000000,6.000000
75%,169340.000000,-0.271116,3.000000,2.000000,1.000000,6.000000,6.000000
max,172320.000000,-0.240029,3.000000,8.000000,7.000000,7.000000,9.000000
